# **Tarefa 4**: Álgebra Linear e Otimização para ML -  MO431A
Universidade Estadual de Campinas (UNICAMP), Instituto de Computação (IC)

Prof. Jacques Wainer, 2021s1


In [1]:
# RA & Name
print('265673: ' + 'Gabriel Luciano Gomes')
print('192880: ' + 'Lucas Borges Rondon')
print('265674: ' + 'Paulo Júnio Reis Rodrigues')

265673: Gabriel Luciano Gomes
192880: Lucas Borges Rondon
265674: Paulo Júnio Reis Rodrigues


## Imports necessários para a tarefa

In [2]:
import numpy as np

from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error

from scipy.stats import loguniform, uniform

from hyperopt import hp, tpe, fmin, STATUS_OK

from pyswarm import pso

from cma import CMAEvolutionStrategy

from simanneal import Annealer

## Leitura da base de dados

In [3]:
X = np.load('db/X.npy')
Y = np.load('db/y.npy')

## Variáveis Globais

In [4]:
# C lower and upper bounds
c_lb = -5
c_ub = 15

# C lower and upper bounds
g_lb = -15
g_ub = 3

#epsilon lower and upper bounds
e_lb = 0.05
e_ub = 1.0

# Base SVM model
base_model = SVR(kernel = 'rbf')

## Funções úteis

### Computar RMSE

In [5]:
def compute_rmse(scores):
    # Compute RMSE
    return np.sqrt(np.mean(np.absolute(scores)))  

### Calcular cross val score

In [6]:
def hyperopt_train_test(params): 
    ''' Computes the cross validation score
    to be compared in order to identify
    the best value
    @params: list of params to SVR (C, gamma, epsilon and Kernel)
    '''
    clf = SVR(**params)
    return cross_val_score(clf, X, Y).mean()

### SVM Regressor

In [7]:
def compute_SVM_result(c, gamma, epsilon):    
    # define cross validation score
    cv = KFold(n_splits = 5, random_state = 1, shuffle = True)

    # Compute SVM
    svr = SVR(kernel = 'rbf', C = c, gamma = gamma, epsilon = epsilon)

    # SVM scores
    scores = cross_val_score(svr, X, Y, scoring = ('neg_mean_squared_error'), cv = cv) 
    
    show_results(c, gamma, epsilon, compute_rmse(scores))
    

### Exibir resultados

In [8]:
def show_results(c, gamma, epsilon, rmse) :
    print('----- Best values of hyperparameters ----- \n' +
      f'C: {round(c, 6)}\ngamma: {round(gamma, 6)} \nepsilon: {round(epsilon, 6)} \n' +
      '----- RMSE for given values ----- \n' + 
      f'RMSE: {round(rmse, 6)}')

## Random Search

In [9]:
# Search space
space = dict()
space['C'] = loguniform(2**c_lb, 2**c_ub)
space['gamma'] = loguniform(2**g_lb, 2**g_ub)
space['epsilon'] = uniform(e_lb, e_ub)

# define search
search = RandomizedSearchCV(base_model,
                            space,
                            n_iter = 125,
                            scoring = 'neg_mean_squared_error',
                            n_jobs = -1,
                            cv = 5,
                            random_state = 1)
result = search.fit(X, Y)
c = result.best_params_['C']
g = result.best_params_['gamma']
e = result.best_params_['epsilon']

### Resultados obtidos

In [10]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 8584.928547
gamma: 3.2e-05 
epsilon: 0.623679 
----- RMSE for given values ----- 
RMSE: 4.023489


## Grid Search

In [11]:
# grid size
g_size = 5

# Search space
space = dict()
space['C'] = loguniform.rvs(2**c_lb, 2**c_ub, size = g_size) 
space['gamma'] = loguniform.rvs(2**g_lb, 2**g_ub, size = g_size)
space['epsilon'] = uniform.rvs(e_lb, e_ub, size = g_size)

# define search
search = GridSearchCV(base_model,
                      space,
                      scoring = 'neg_mean_squared_error',
                      n_jobs = -1,
                      cv = 5)

result = search.fit(X, Y)
c = result.best_params_['C']
e = result.best_params_['epsilon']
g = result.best_params_['gamma']

### Resultados obtidos

In [12]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 2145.208037
gamma: 5.9e-05 
epsilon: 0.276161 
----- RMSE for given values ----- 
RMSE: 4.326304


## Bayesian Optimization

In [13]:
def objective_function_bo(params):
    ''' Callable function to compare SVR scores.
    For this example, loss will be used.
    @params: list of params to SVR (C, gamma, epsilon and Kernel)
    '''   
    C =  params['C']
    gamma = params['gamma']
    epsilon = params['epsilon']
    acc = hyperopt_train_test({'C': 2**C, 'gamma': 2**gamma, 'epsilon': epsilon})
    
    return {'loss': -acc, 'status': STATUS_OK}

In [14]:
space = {
    'C': hp.uniform('C', c_lb, c_ub),
    'gamma': hp.uniform('gamma', g_lb, g_ub),
    'epsilon': hp.uniform('epsilon', e_lb, e_ub)   
}

best = fmin(objective_function_bo, space, algo = tpe.suggest, max_evals = 125)
c = 2** best['C']
e = 2** best['epsilon']
g = 2** best['gamma']

100%|█████████████████████████████████████████████| 125/125 [03:33<00:00,  1.71s/trial, best loss: -0.8276213557206253]


### Resultados obtidos

In [15]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 20081.964026
gamma: 3.2e-05 
epsilon: 1.309902 
----- RMSE for given values ----- 
RMSE: 3.976988


## PSO

In [16]:
def objective_function_pso(x):
    C, gamma, epsilon = x
    kernel =  'rbf'
    acc = hyperopt_train_test({'C': 2**C, 'gamma': 2**gamma, 'epsilon': epsilon, 'kernel': kernel})
    return -acc

In [17]:
# upper and lower bounds for C, gamma and epsilon respectively
lb = [c_lb, g_lb, e_lb]
ub = [c_ub, g_ub, e_ub]

xopt, fopt = pso(objective_function_pso, lb, ub, swarmsize = 11, maxiter = 11)

c = 2** xopt[0]
g = 2** xopt[1]
e = xopt[2]

Stopping search: maximum iterations reached --> 11


### Resultados obtidos

In [18]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 22121.916813
gamma: 3.1e-05 
epsilon: 0.180888 
----- RMSE for given values ----- 
RMSE: 4.101285


## Simulated Annealing

Classe Filha do Annealing, necessária para funcionamento

In [19]:
class SimulatedAnnealing(Annealer):
    """Test annealer to objetctive function"""
    
    def __init__(self, state):
        super(SimulatedAnnealing, self).__init__(state)
    
    def move(self):
        """Swaps params of SVM."""
        self.state[0] = 2 ** np.random.uniform(low = c_lb, high = c_ub)
        self.state[1] = 2 ** np.random.uniform(low = g_lb, high = g_ub)
        self.state[2] = np.random.uniform(low = e_lb, high = e_ub)
        
    def energy(self):
        """Calculates cross validation score"""
        C, gamma, epsilon = self.state[0], self.state[1], self.state[2]
        kernel =  'rbf'
        
        return self.objective_function_sa({
            'C': C,
            'gamma': gamma,
            'epsilon': epsilon,
            'kernel': kernel
        })
    
    def objective_function_sa(self, x):        
        acc = hyperopt_train_test(x)        
        return -acc

In [20]:
initial_state = [
      2 ** np.random.uniform(low = c_lb, high = c_ub),
      2 ** np.random.uniform(low = g_lb, high = g_ub),
      np.random.uniform(low = e_lb, high = e_ub)    
]

sa = SimulatedAnnealing(initial_state)
sa.steps = 125

xopt, fopt = sa.anneal()
c = xopt[0]
g = xopt[1]
e = xopt[2]

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         -0.79     0.00%     0.00%     0:00:28     0:00:00

### Resultados obtidos

In [21]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 2715.058986
gamma: 6.6e-05 
epsilon: 0.393088 
----- RMSE for given values ----- 
RMSE: 4.30675


## CMA-ES

In [22]:
def objetive_function_CMA_ES(x):
    C, gamma, epsilon = x
    kernel =  'rbf'
    acc = hyperopt_train_test({'C': 2** (c_lb + C*20),
                               'gamma': 2** (g_lb + gamma*18),
                               'epsilon': abs(epsilon),
                               'kernel': kernel})
    return -acc

In [23]:
# Define initial bounds
lw = [0.0, 0.0, 0.0]
up = [1.0, 1.0, 1.0]

# Initial values
x0 = 3 * [0.05]
sigma = 0.25

result = CMAEvolutionStrategy(x0, sigma, {'bounds': [lw, up]})
result.optimize(objetive_function_CMA_ES, iterations = 125)

# extract best hyperparameters values
c = 2 ** (c_lb + result.best.x[0] * 20)
g = 2 ** (g_lb + result.best.x[1] * 18)
e = abs(result.best.x[2])

(3_w,7)-aCMA-ES (mu_w=2.3,w_1=58%) in dimension 3 (seed=692023, Tue Apr 27 21:22:40 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      7 -2.315199708422314e-01 1.0e+00 2.08e-01  2e-01  2e-01 0:00.6
    2     14 -3.320906359970631e-01 1.2e+00 2.14e-01  2e-01  2e-01 0:01.1
    3     21 -6.616681538101122e-01 1.5e+00 2.40e-01  2e-01  3e-01 0:01.9
    4     28 -4.394857983716051e-01 1.7e+00 2.65e-01  2e-01  3e-01 0:10.7
    5     35 -8.039246495667275e-01 1.6e+00 4.26e-01  3e-01  5e-01 0:34.0
    6     42 -7.403545289565399e-01 1.7e+00 4.57e-01  3e-01  5e-01 0:47.5
    7     49 -8.196302081259074e-01 1.8e+00 4.89e-01  3e-01  5e-01 1:00.1
    9     63 -8.233516654906345e-01 2.5e+00 5.97e-01  3e-01  8e-01 1:25.2
   10     70 -7.586400226082639e-01 3.1e+00 5.58e-01  3e-01  7e-01 1:38.4
   11     77 -6.634747558410108e-01 3.2e+00 5.08e-01  2e-01  6e-01 1:52.1
   12     84 -8.198820025351490e-01 3.2e+00 5.06e-01  2e-01  6e-01 2:09.0
   13     91 -8.29597411

### Resultados obtidos

In [24]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 23682.252806
gamma: 3.1e-05 
epsilon: 0.001736 
----- RMSE for given values ----- 
RMSE: 4.210726
